<a href="https://colab.research.google.com/github/abosedealli/Stockprediction/blob/main/LINEAR%20REGRESSION%20stock_dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install tensorflow
import tensorflow as tf
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing the libraries to Predict the closing price stock price 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import History 
import seaborn as sns
import keras
from keras.callbacks import History 
from sklearn.metrics import mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
sns.set_style('whitegrid')
from datetime import datetime

## Stocks to use for analysis

---

Apple dataset from inception till till 9/1/2023 in US currency except date and volume.



In [4]:
data = pd.read_csv("/content/apple.dataset.csv")


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       10100 non-null  object 
 1   High       10100 non-null  float64
 2   Low        10100 non-null  float64
 3   Open       10100 non-null  float64
 4   Close      10100 non-null  float64
 5   Volume     10100 non-null  int64  
 6   Adj Close  10100 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 552.5+ KB


In [6]:
data.set_index(pd.DatetimeIndex(data['Date']), inplace=True)

In [7]:
data

,Date,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
1980-12-12,12/12/1980,0.128906,0.128348,0.128348,0.128348,469033600,0.100039
1980-12-15,15/12/1980,0.122210,0.121652,0.122210,0.121652,175884800,0.094820
1980-12-16,16/12/1980,0.113281,0.112723,0.113281,0.112723,105728000,0.087861
1980-12-17,17/12/1980,0.116071,0.115513,0.115513,0.115513,86441600,0.090035
1980-12-18,18/12/1980,0.119420,0.118862,0.118862,0.118862,73449600,0.092646
...,...,...,...,...,...,...,...
2020-12-24,24/12/2020,133.460007,131.100006,131.320007,131.970001,54930100,130.620911
2020-12-28,28/12/2020,137.339996,133.509995,133.990005,136.690002,124486200,135.292664
2020-12-29,29/12/2020,138.789993,134.339996,138.050003,134.869995,121047300,133.491242


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10100 entries, 1980-12-12 to 2020-12-31
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       10100 non-null  object 
 1   High       10100 non-null  float64
 2   Low        10100 non-null  float64
 3   Open       10100 non-null  float64
 4   Close      10100 non-null  float64
 5   Volume     10100 non-null  int64  
 6   Adj Close  10100 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 631.2+ KB


In [9]:
apple= data.assign(**{'average': lambda df: df.loc[:,['Open','High','Low','Close']].mean(axis=1), 
                    'EMA20': lambda df: df['average'].ewm(span=20, adjust=False).mean(), 
                    'EMA5': lambda df: df['average'].ewm(span=5, adjust=False).mean(), 
                    'dist_EMA20': lambda df: (df['average'] - df['EMA20'])/df['EMA20']*100, 
                    'dist_EMA5': lambda df: (df['average'] - df['EMA5'])/df['EMA5']*100})
apple.head()

,Date,High,Low,Open,Close,Volume,Adj Close,average,EMA20,EMA5,dist_EMA20,dist_EMA5
Date,,,,,,,,,,,,
1980-12-12,12/12/1980,0.128906,0.128348,0.128348,0.128348,469033600,0.100039,0.128487,0.128487,0.128487,0.000000,0.000000
1980-12-15,15/12/1980,0.122210,0.121652,0.122210,0.121652,175884800,0.094820,0.121931,0.127863,0.126302,-4.639389,-3.460749
1980-12-16,16/12/1980,0.113281,0.112723,0.113281,0.112723,105728000,0.087861,0.113002,0.126448,0.121869,-10.633428,-7.275591
1980-12-17,17/12/1980,0.116071,0.115513,0.115513,0.115513,86441600,0.090035,0.115652,0.125420,0.119797,-7.787547,-3.459289
1980-12-18,18/12/1980,0.119420,0.118862,0.118862,0.118862,73449600,0.092646,0.119002,0.124808,0.119532,-4.652620,-0.443457


In [10]:
apple.columns

Index(['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close',
       'average', 'EMA20', 'EMA5', 'dist_EMA20', 'dist_EMA5'],
      dtype='object')

In [19]:
apple

,Date,High,Low,Open,Close,Volume,Adj Close,average,EMA20,EMA5,dist_EMA20,dist_EMA5
Date,,,,,,,,,,,,
1980-12-12,12/12/1980,0.128906,0.128348,0.128348,0.128348,469033600,0.100039,0.128487,0.128487,0.128487,0.000000,0.000000
1980-12-15,15/12/1980,0.122210,0.121652,0.122210,0.121652,175884800,0.094820,0.121931,0.127863,0.126302,-4.639389,-3.460749
1980-12-16,16/12/1980,0.113281,0.112723,0.113281,0.112723,105728000,0.087861,0.113002,0.126448,0.121869,-10.633428,-7.275591
1980-12-17,17/12/1980,0.116071,0.115513,0.115513,0.115513,86441600,0.090035,0.115652,0.125420,0.119797,-7.787547,-3.459289
1980-12-18,18/12/1980,0.119420,0.118862,0.118862,0.118862,73449600,0.092646,0.119002,0.124808,0.119532,-4.652620,-0.443457
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,24/12/2020,133.460007,131.100006,131.320007,131.970001,54930100,130.620911,131.962505,125.430239,130.265986,5.207888,1.302350
2020-12-28,28/12/2020,137.339996,133.509995,133.990005,136.690002,124486200,135.292664,135.382500,126.378074,131.971490,7.124991,2.584656
2020-12-29,29/12/2020,138.789993,134.339996,138.050003,134.869995,121047300,133.491242,136.512497,127.343257,133.485159,7.200413,2.267921


In [20]:
apple_model_data=apple[['Adj Close', 'High', 'EMA20', 'Close']]
# apple.columns

In [21]:
apple_model_data.head()

,Adj Close,High,EMA20,Close
Date,,,,
1980-12-12,0.100039,0.128906,0.128487,0.128348
1980-12-15,0.094820,0.122210,0.127863,0.121652
1980-12-16,0.087861,0.113281,0.126448,0.112723
1980-12-17,0.090035,0.116071,0.125420,0.115513
1980-12-18,0.092646,0.119420,0.124808,0.118862


In [22]:
X = apple_model_data[['Adj Close', 'EMA20', 'High']]
Y = apple_model_data['Close']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 99)


In [26]:
X_train.head()

,Adj Close,EMA20,High
Date,,,
2007-07-24,4.113336,4.765145,5.035714
2017-05-10,36.868813,38.882101,38.860001
2016-08-19,25.497681,26.556005,27.422501
1988-09-21,0.300746,0.367531,0.383929
1994-06-12,0.283125,0.344575,0.342634


In [29]:
print(X_test.describe())

         Adj Close        EMA20         High
count  2020.000000  2020.000000  2020.000000
mean      9.002100     9.504533     9.731303
std      18.459168    18.618458    19.217086
min       0.040016     0.055964     0.051897
25%       0.225654     0.273178     0.272321
50%       0.364880     0.434183     0.445246
75%       9.383309    10.384909    11.062946
max     127.384315   122.363731   129.580002


In [30]:
print(X_train.describe())

         Adj Close        EMA20         High
count  8080.000000  8080.000000  8080.000000
mean      9.354274     9.883538    10.110898
std      19.135987    19.304992    19.899067
min       0.038276     0.055824     0.049665
25%       0.229010     0.276265     0.281250
50%       0.368176     0.444663     0.450893
75%       9.915427    11.482358    11.641517
max     135.292664   128.542825   138.789993


In [31]:
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
  return mape

In [32]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train, y_train)

# Make the prediction
y_pred = reg.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape= mean_absolute_percentage_error(y_test, y_pred)
print('Mean abosulte percentage error:', mape)

Mean abosulte percentage error: 11.363125874814292


In [33]:
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R Square value:', r2)

Mean Absolute Error: 0.10453893285123146
Mean Squared Error: 0.05885095322339749
R Square value: 0.9998369722551902


Predicting to check for short term

In [34]:
#Nexflix dataset from 2020 till Dec 2022

In [40]:
n_data = pd.read_csv("/content/neflix.csv")

In [41]:
n_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2/01/2020,326.100006,329.980011,324.779999,329.809998,329.809998,4485800
1,3/01/2020,326.779999,329.859985,325.529999,325.899994,325.899994,3806900
2,6/01/2020,323.119995,336.359985,321.200012,335.829987,335.829987,5663100
3,7/01/2020,336.470001,336.700012,330.299988,330.750000,330.750000,4703200
4,8/01/2020,331.489990,342.700012,331.049988,339.260010,339.260010,7104500
...,...,...,...,...,...,...,...
751,23/12/2022,296.179993,298.459991,291.910004,294.959991,294.959991,4251100
752,27/12/2022,293.190002,293.570007,282.130005,284.170013,284.170013,5778100
753,28/12/2022,281.920013,285.190002,273.410004,276.880005,276.880005,5964400
754,29/12/2022,283.179993,295.500000,281.010010,291.119995,291.119995,9588500


In [42]:
Nexflix= n_data.assign(**{'average': lambda df: df.loc[:,['Open','High','Low','Close']].mean(axis=1), 
                    'EMA20': lambda df: df['average'].ewm(span=20, adjust=False).mean(), 
                    'EMA5': lambda df: df['average'].ewm(span=5, adjust=False).mean(), 
                    'dist_EMA20': lambda df: (df['average'] - df['EMA20'])/df['EMA20']*100, 
                    'dist_EMA5': lambda df: (df['average'] - df['EMA5'])/df['EMA5']*100})

In [43]:
Nexflix

,Date,Open,High,Low,Close,Adj Close,Volume,average,EMA20,EMA5,dist_EMA20,dist_EMA5
0,2/01/2020,326.100006,329.980011,324.779999,329.809998,329.809998,4485800,327.667504,327.667504,327.667504,0.000000,0.000000
1,3/01/2020,326.779999,329.859985,325.529999,325.899994,325.899994,3806900,327.017494,327.605598,327.450834,-0.179516,-0.132337
2,6/01/2020,323.119995,336.359985,321.200012,335.829987,335.829987,5663100,329.127495,327.750540,328.009721,0.420123,0.340775
3,7/01/2020,336.470001,336.700012,330.299988,330.750000,330.750000,4703200,333.555000,328.303346,329.858147,1.599635,1.120740
4,8/01/2020,331.489990,342.700012,331.049988,339.260010,339.260010,7104500,336.125000,329.048266,331.947098,2.150668,1.258605
...,...,...,...,...,...,...,...,...,...,...,...,...
751,23/12/2022,296.179993,298.459991,291.910004,294.959991,294.959991,4251100,295.377495,298.255081,295.042102,-0.964807,0.113676
752,27/12/2022,293.190002,293.570007,282.130005,284.170013,284.170013,5778100,288.265007,297.303646,292.783070,-3.040205,-1.543144
753,28/12/2022,281.920013,285.190002,273.410004,276.880005,276.880005,5964400,279.350006,295.593775,288.305382,-5.495302,-3.106212
754,29/12/2022,283.179993,295.500000,281.010010,291.119995,291.119995,9588500,287.702499,294.842225,288.104421,-2.421541,-0.139506


In [80]:
Nexflix_model_data=Nexflix[['Adj Close', 'High', 'EMA20']]

Y2 = Nexflix['Close']

In [82]:
X2 = Nexflix[['Adj Close','High', 'EMA20']]

In [83]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X2, Y2, test_size=0.2, random_state = 99)

In [84]:
print(Xtest.describe())
print(Xtrain.describe())

        Adj Close        High       EMA20
count  152.000000  152.000000  152.000000
mean   435.659276  442.529670  434.158037
std    134.009562  135.167110  131.536781
min    173.100006  182.759995  181.306901
25%    338.429993  342.967506  337.825008
50%    480.455001  491.524994  487.468683
75%    525.972504  533.349991  522.199071
max    691.690002  700.989990  667.736024
        Adj Close        High       EMA20
count  604.000000  604.000000  604.000000
mean   428.712566  435.627532  429.529734
std    134.491372  135.294833  131.285829
min    166.369995  172.059998  181.221575
25%    331.775002  339.727501  334.108351
50%    480.540008  486.599991  489.014658
75%    524.842514  533.509994  521.971391
max    690.309998  694.159973  667.730079


In [85]:
def mean_absolute_percentage_error(y_truth, y_Pred):
  y_truth, y_Pred = np.array(y_truth), np.array(y_Pred)
  mape = np.mean(np.abs((y_truth - y_Pred) / y_truth))*100
  return mape

In [87]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(Xtrain, Ytrain)

# Make the prediction
y_Pred = reg.predict(Xtest)

# Evaluate the model
mse = mean_squared_error(Ytest, y_Pred)
r2 = r2_score(Ytest, y_Pred)
mape= mean_absolute_percentage_error(Ytest, y_Pred)
print('Mean abosulte percentage error:', mape)

Mean abosulte percentage error: 2.945598159001199e-14


In [88]:
mae = mean_absolute_error(Ytest, y_Pred)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R Square value:', r2)

Mean Absolute Error: 7.778573107268465e-14
Mean Squared Error: 1.5103613929351514e-26
R Square value: 1.0
